In [1]:
import __init__
#
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
def regression(dv, df):
    oc_dv = 'roamingTime' if dv == 'spendingTime' else 'spendingTime'
    rdf = df.copy(deep=True).drop([oc_dv], axis=1)
#     rdf = rdf[~(np.abs(rdf[dv] - rdf[dv].mean()) > (3 * rdf[dv].std()))]
    candi_dummies = []
    num_iter = 1
    while True:
        for i, vs in enumerate(zip(*rdf.values)):
            if rdf.columns[i] == dv:
                continue
            if sum(vs) > len(rdf) * MIN_PICKUP_RATIO * num_iter:
                candi_dummies.append(rdf.columns[i])
        if len(rdf) <= len(candi_dummies):
            candi_dummies = []
            num_iter += 1
        else:
            break
    y = rdf[dv]
    X = rdf[candi_dummies]
    X = sm.add_constant(X)
    return sm.OLS(y, X, missing='drop').fit()

In [3]:
from community_analysis import tfZ_TP_dpath, tfZ_TP_prefix
#
df = pd.read_csv('%s/%s2009-3.csv' % (tfZ_TP_dpath, tfZ_TP_prefix))

In [9]:
df

month  day  timeFrame  zi  zj         tfZ    did  spendingTime  \
0          1    1         14  54  12  (14,54,12)  36942           240   
1          1    1         14  38  45  (14,38,45)  33720            48   
2          1    1         14  61  27  (14,61,27)  20677           592   
3          1    1         14  53  25  (14,53,25)  29328           596   
4          1    1         14  41  16  (14,41,16)  19360           356   
5          1    1         14  55  12  (14,55,12)  36942            34   
6          1    1         14  53  25  (14,53,25)  37230            15   
7          1    1         14  29  18  (14,29,18)  31043            27   
8          1    1         14  59  30  (14,59,30)   1098             3   
9          1    1         14  75  25  (14,75,25)  13781            40   
10         1    1         14  36  12  (14,36,12)  11382            45   
11         1    1         14  51  28  (14,51,28)  14664            18   
12         1    1         14  29  18  (14,29,18)  31043            83   
13         1    1         14  49  12  (14,49,12)  19360            64   
14         1    1         15  64  17  (15,64,17)  33720           723   
15         1    1         15  57  14  (15,57,14)  13781           238   
16         1    1         15  19  23  (15,19,23)  11382            50   
17         1    1         15  50   9   (15,50,9)    813            22   
18         1    1         15  51  12  (15,51,12)  19360           120   
19         1    1         15  25  25  (15,25,25)   3145            46   
20         1    1         15  36  45  (15,36,45)  33650          1080   
21         1    1         15  54  16  (15,54,16)  19360            23   
22         1    1         15  47   6   (15,47,6)   1098           386   
23         1    1         15  76  24  (15,76,24)  21597            61   
24         1    1         15  19  23  (15,19,23)  33720            35   
25         1    1         15  19  22  (15,19,22)   3145             3   
26         1    1         15  34  31  (15,34,31)  33650           127   
27         1    1         15  68  15  (15,68,15)  36942            24   
28         1    1         15  52  28  (15,52,28)  34978            69   
29         1    1         15  53  10  (15,53,10)   1537           107   
...      ...  ...        ...  ..  ..         ...    ...           ...   
47325     11   30         21  34  41  (21,34,41)  23456            23   
47326     11   30         21  41  16  (21,41,16)  21812            80   
47327     11   30         21  75  25  (21,75,25)  15346           157   
47328     11   30         21  79  21  (21,79,21)  28615            27   
47329     11   30         22  43  45  (22,43,45)  33720           563   
47330     11   30         22  60  20  (22,60,20)   6361            45   
47331     11   30         22  79  21  (22,79,21)  28615            49   
47332     11   30         22  79  21  (22,79,21)   4629            27   
47333     11   30         22  54  21  (22,54,21)  15346            99   
47334     11   30         22  66  18  (22,66,18)   6361            29   
47335     11   30         22  37  46  (22,37,46)  33720             3   
47336     11   30         22  78  27  (22,78,27)   1799            27   
47337     11   30         22  85  26  (22,85,26)   4009           716   
47338     11   30         22  75  24  (22,75,24)   6361            61   
47339     11   30         23  60  20  (23,60,20)   4629            15   
47340     11   30         23  85  27  (23,85,27)  17877            25   
47341     11   30         23  46  15  (23,46,15)   4009            10   
47342     11   30         23  84  26  (23,84,26)  20393            39   
47343     11   30         23  52  29  (23,52,29)  13781             6   
47344     11   30         23  56  14  (23,56,14)   1799             2   
47345     11   30         23  83  22  (23,83,22)  46139           117   
47346     11   30         23  55  15  (23,55,15)   4009             6   
47347     11   30         23  48  14  (23,48,14)  17877          

In [10]:
did1 = 31043
did1_df = df[(df['did'] == did1)].copy(deep=True)
did1_df = did1_df.drop(['month', 'day', 'timeFrame', 'zi', 'zj', 'tfZ', 'did'], axis=1)
if '%d' % did1 in did1_df.columns:
    did1_df = did1_df.drop(['%d' % did1], axis=1)

In [11]:
len(did1_df), len(did1_df.columns)

(149, 6105)

In [12]:
MIN_PICKUP_RATIO = 0.1
SP_res = regression('spendingTime', did1_df)

In [13]:
did1_df['spendingTime'].mean()

91.738255033557053

In [14]:
print SP_res.summary()

                            OLS Regression Results                            
Dep. Variable:           spendingTime   R-squared:                       0.424
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     2.900
Date:                Mon, 16 Jan 2017   Prob (F-statistic):           2.31e-05
Time:                        14:39:52   Log-Likelihood:                -931.17
No. Observations:                 149   AIC:                             1924.
Df Residuals:                     118   BIC:                             2017.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         74.4381     14.742      5.049      0.0

In [15]:
RP_res = regression('roamingTime', did1_df)

In [16]:
print RP_res.summary()

                            OLS Regression Results                            
Dep. Variable:            roamingTime   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     2.160
Date:                Mon, 16 Jan 2017   Prob (F-statistic):            0.00186
Time:                        14:40:18   Log-Likelihood:                -1125.0
No. Observations:                 149   AIC:                             2312.
Df Residuals:                     118   BIC:                             2405.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        874.3205     54.134     16.151      0.0